In [1]:
import time
import torch
import torchvision
import torchvision.models as models
import torch._dynamo as dynamo

/Users/chenzhongming/anaconda3/lib/python3.7/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/chenzhongming/anaconda3/lib/python3.7/site-packages/torchvision/image.so, 6): Symbol not found: __ZN3c106detail19maybe_wrap_dim_slowExxb
  Referenced from: /Users/chenzhongming/anaconda3/lib/python3.7/site-packages/torchvision/image.so
  Expected in: /Users/chenzhongming/anaconda3/lib/python3.7/site-packages/torch/lib/libc10.dylib
 in /Users/chenzhongming/anaconda3/lib/python3.7/site-packages/torchvision/image.so
  warn(f"Failed to load image Python extension: {e}")


In [2]:
print(torch.__version__)

1.14.0.dev20221205


In [3]:
def foo(x, y):
    a = torch.sin(x)
    b = torch.cos(x)
    return a + b

In [5]:
start = time.time()
compiled_model = torch.compile(foo)
out = compiled_model(torch.randn(10, 10), torch.randn(10, 10))
end = time.time()
print(end - start)

0.002129793167114258


In [6]:
start = time.time()
opt_foo1 = dynamo.optimize("inductor")(foo)
result = opt_foo1(torch.randn(10, 10), torch.randn(10, 10))
end = time.time()
print(end - start)

0.0015070438385009766


In [7]:
start = time.time()
result = foo(torch.randn(10, 10), torch.randn(10, 10))
end = time.time()
print(end - start)

0.0008828639984130859


In [41]:
model = models.alexnet()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
compiled_model = torch.compile(model)

x = torch.randn(16, 3, 224, 224)
optimizer.zero_grad()

start = time.time()
out = compiled_model(x)
out.sum().backward()
optimizer.step()
end = time.time()
print(end - start)

/Users/chenzhongming/anaconda3/lib/python3.7/site-packages/torch/storage.py:315: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.
  warnings.warn(message, UserWarning)
/Users/chenzhongming/anaconda3/lib/python3.7/site-packages/torch/storage.py:315: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.
  warnings.warn(message, UserWarning)
[2022-12-06 00:27:04,313] torch._inductor.lowering: [WARNING] using triton random, expect difference from eager
[2022-12-06 00:27:04,868] torch._inductor.ir: [WARNING] Using FallbackKernel: aten._adaptive_avg_pool2d_backward


1.8354921340942383


In [42]:
start = time.time()
out = model(x)
out.sum().backward()
optimizer.step()
end = time.time()
print(end - start)

0.9873549938201904


In [47]:
count = []

for epoch in range(10):
    start = time.time()
    out = model(x)
    out.sum().backward()
    optimizer.step()
    end = time.time()
    count.append(end - start)
    print('Finished Training:', end - start)

print(sum(count)/len(count))

Finished Training: 0.9222769737243652
Finished Training: 1.017333984375
Finished Training: 1.0016257762908936
Finished Training: 0.9622230529785156
Finished Training: 1.031148910522461
Finished Training: 0.9600911140441895
Finished Training: 0.9932949542999268
Finished Training: 1.0256049633026123
Finished Training: 1.0021679401397705
Finished Training: 1.0009710788726807
0.9916738748550415


In [46]:
count = []

for epoch in range(10):
    start = time.time()
    out = compiled_model(x)
    out.sum().backward()
    optimizer.step()
    end = time.time()
    count.append(end - start)
    print('Finished Training:', end - start)
    
print(sum(count)/len(count))

Finished Training: 0.9238681793212891
Finished Training: 0.9480807781219482
Finished Training: 0.9517581462860107
Finished Training: 0.9673218727111816
Finished Training: 0.9698529243469238
Finished Training: 0.975740909576416
Finished Training: 0.9405207633972168
Finished Training: 0.9825499057769775
Finished Training: 0.8716769218444824
Finished Training: 0.9171638488769531
0.9448534250259399
